In [49]:
import numpy as np
import itertools
import random
from statistics import stdev
import math
import ast

In [50]:

p_running = 0.9
p_failure = 1 - p_running
path = '/home/dkabe/Model_brainstorming/SAA_Analysis_v6/Instance_2/'

In [51]:
N = [64, 128, 192, 256, 320, 384, 448]
N = [192, 256, 320, 384]
batches = 30
Products = 3
Market = 29
text_file = open(path + 'Disruption_scenarios.txt', "r")
ls = text_file.read().split('\n')[:-1]
Scenarios = list(map(lambda x: ast.literal_eval(x), ls))
Probabilities = np.loadtxt(path + "disruption_probabilities.txt")
num_disruption = len(Scenarios)

In [52]:
p1_mean = 137000
p2_mean = 135000
p3_mean = 133000

p1_sd = 2650
p2_sd = 8125
p3_sd = 4000

distribution_mat = [[p1_mean, p1_sd],
                  [p2_mean, p2_sd],
                  [p3_mean, p3_sd]]

In [53]:
np.random.seed(2)
nominal_demand = np.zeros((Products, Market))
for m in range(Products):
    nominal_demand[m] = np.round(np.random.normal(distribution_mat[0][0], distribution_mat[0][1], size = 29))
    nominal_demand[m] = np.round(np.random.normal(distribution_mat[1][0], distribution_mat[1][1], size = 29))
    nominal_demand[m] = np.round(np.random.normal(distribution_mat[2][0], distribution_mat[2][1], size = 29))

In [54]:
delta = np.arange(0.75, 1.255, 0.005)

In [55]:
demand_realization = len(delta)


In [56]:
random.seed(2)
demand = np.zeros((demand_realization, Products, Market))
for r in range(demand_realization):
    demand[r] = delta[r]*nominal_demand

In [57]:
S = demand_realization*Scenarios
S_demand = []
S_prob = list(Probabilities)*demand_realization
multiplier = sum(S_prob)
S_prob = list(map(lambda x: x/multiplier, S_prob))
for r in range(demand_realization):
    S_demand+=([demand[r]]*num_disruption)

In [58]:
len(S_demand)

103424

In [264]:
#random.seed(20)
evaluation_indices = random.sample(range(len(S)), 10000)
evaluation_set = [S[index] for index in evaluation_indices]
evaluation_demand = np.array([S_demand[index] for index in evaluation_indices])
eval_probabilities = [S_prob[index] for index in evaluation_indices]
p_factor = sum(eval_probabilities)
eval_probabilities = list(map(lambda x: x/p_factor, eval_probabilities))
with open(path + 'Evaluation_Set.txt', 'w+') as f:
    for item in evaluation_set:
        f.write("%s\n" % item)
f.close()
np.savetxt(path + "p_scen.txt", eval_probabilities)
np.savetxt(path + "eval_set_demand.txt", evaluation_demand.reshape((10000*3,29)))

In [226]:
S[scen][0]

[1, 1, 0, 0, 1, 1]

In [277]:
S_filtered = []
#list(filter(lambda x: np.sum(x[0]) >= 2 and np.sum(x[1]) >= 2, S))
d_filtered = [] 
p_filtered = []
for scen in range(len(S)):
    if np.sum(S[scen][0]) >= 3 and np.sum(S[scen][1]) >= 2 and np.sum(S_demand[scen]) <= 1.1*np.sum(nominal_demand):
        S_filtered.append(S[scen])
        d_filtered.append(S_demand[scen])
        p_filtered.append(S_prob[scen])
        


In [279]:
d_filtered

KeyboardInterrupt: 

In [280]:
evaluation_indices = random.sample(range(len(S_filtered)), 10000)
evaluation_set = [S_filtered[index] for index in evaluation_indices]
evaluation_demand = np.array([d_filtered[index] for index in evaluation_indices])
eval_probabilities = [p_filtered[index] for index in evaluation_indices]
p_factor = sum(eval_probabilities)
eval_probabilities = list(map(lambda x: x/p_factor, eval_probabilities))
with open(path + 'Evaluation_Set.txt', 'w+') as f:
    for item in evaluation_set:
        f.write("%s\n" % item)
f.close()
np.savetxt(path + "p_scen.txt", eval_probabilities)
np.savetxt(path + "eval_set_demand.txt", evaluation_demand.reshape((10000*3,29)))

In [283]:
np.mean(nominal_demand)

132825.54022988505

In [266]:

random.seed(4)
for num_scen in N:
    for batch in range(batches):
        indices = random.sample(range(len(S)), num_scen)
        sample_scenarios = [S[index] for index in indices]
        p_sample_scenarios = [S_prob[index] for index in indices]
        demand = np.array([S_demand[index] for index in indices])
        p_factor = 1/np.sum(p_sample_scenarios)
        p_sample_scenarios = list(map(lambda x: x*p_factor, p_sample_scenarios))  
        with open(path + 'Scenarios/' + str(num_scen) + '_' + str(batch) + '.txt', 'w') as f:
            for item in sample_scenarios:
                f.write("%s\n" % item)
        f.close()
        np.savetxt(path + 'Scen_probabilities/' + 'p_scen_' + str(num_scen) + '_' + str(batch) + '.txt', p_sample_scenarios)
        np.savetxt(path + 'Scen_demand/' + 'demand_' + str(num_scen) + '_' + str(batch) + '.txt', demand.reshape((num_scen*Products, Market)))

In [297]:
# version 2
batches=30
path = '/home/dkabe/Model_brainstorming/SAA_Analysis_v6/Instance_2/'
N = [64, 128, 192, 256, 320]#, 384]#, 448]
N = [192]
p_scen = np.loadtxt('/home/dkabe/Model_brainstorming/SAA_Analysis_v6/Instance_2/p_scen.txt')
p_factor = 1/np.sum(p_scen)
p_scen = list(map(lambda x: x*p_factor, p_scen))  
for scen in N:
    gaps = []
    ub_averages = []
    ub_max = []
    objvals_lb = np.loadtxt(path + "Objectives/" + str(scen) + '_results.txt')
    avg_lb = (np.mean(objvals_lb))
    std_lb = (stdev(objvals_lb))
    lbwidth = std_lb/math.sqrt(batches)*2.045
    lbmin = round(avg_lb - lbwidth,2)
    #print(avg_lb, lbmin)
    #print('ci on lower bound = [', round(avg_lb-lbwidth,2), ',', round(avg_lb+lbwidth,2), ']')
    for batch in [5]:
        objvals_ub = np.loadtxt(path + "Upper_Bounds/" + str(scen) + "_scenarios/" + str(scen) + "_" + str(batch) + "_UB_results.txt")
        if objvals_ub.ndim == 1:
            avg_ub = (np.sum(p_scen*objvals_ub))
            std_ub = math.sqrt(np.sum(p_scen*((objvals_ub - avg_ub)**2)))
        else:
            avg_ub = np.sum(objvals_ub[:,0]*objvals_ub[:,1])
            std_ub = math.sqrt(np.sum(objvals_ub[:,1]*((objvals_ub[:,0] - avg_ub)**2)))
        ubwidth = std_ub/math.sqrt(10000)*1.96
        ubmax = round(avg_ub + ubwidth, 2)
        gap = round(100*((ubmax - lbmin)/lbmin),2)
        ub_averages.append(avg_ub)
        gaps.append(gap)
        ub_max.append(ubmax)
    #min_gap = min(filter(lambda x: x > 0, gaps))
    min_gap = min(gaps)
    index = gaps.index(min_gap)
    #print(ub_averages[index], ub_max[index], min_gap)
    print(avg_lb, "\t", ub_averages[index], "\t", lbmin, "\t", ub_max[index], "\t", min_gap)


19034302.884 	 17700411.055656992 	 18189351.03 	 17775330.96 	 -2.28


In [259]:
sum(p_scen)

0.9999999999999605

In [10]:
random.seed(4)
indices = random.sample(range(len(S)), 192)
sample_scenarios = [S[index] for index in indices]
p_sample_scenarios = [S_prob[index] for index in indices]
demand = np.array([S_demand[index] for index in indices])
p_factor = 1/np.sum(p_sample_scenarios)
p_sample_scenarios = list(map(lambda x: x*p_factor, p_sample_scenarios))  
#with open("/home/dkabe/Model_brainstorming/Input_Data/Realistic/Instance_2/scen_2.txt", 'w+') as f:
 #   for item in sample_scenarios:
  #      f.write("%s\n" % item)
#f.close()
#np.savetxt("/home/dkabe/Model_brainstorming/Input_Data/Realistic/Instance_2/p_scen_2.txt", p_sample_scenarios)
#np.savetxt("/home/dkabe/Model_brainstorming/Input_Data/Realistic/Instance_2/Demand_2.txt", demand.reshape((192*Products, Market)))

In [142]:
l1 = [192, 256, 320, 384]
for num in l1:
    objvals_lb = np.loadtxt(path + "Objectives/" + str(num) + '_results.txt')
    print(np.mean(objvals_lb))


19034302.884
18923547.886666667
18761562.498666663
19096423.880333338


In [231]:
sum(eval_probabilities)

1.0000000000000158

In [236]:
gaps

[4.87, 3.59, 3.59]